In [7]:
import requests
import pandas as pd
import time
from config import api_key

In [8]:
# Import Location csv file
loc_csv_filepath = "../Transformed Data/location-summary.csv"

In [10]:
# Open csv file
with open(loc_csv_filepath, 'r', encoding='utf-8') as loc_file:
    df_loc = pd.read_csv(loc_file)

df_loc

,country_id,city,state
0,2,Regina,SK
1,2,Vaudreuil-Dorion,QC
2,2,Calgary,AB
3,2,Iqaluit,NU
4,2,Lethbridge,AB
...,...,...,...
1072,1,North-East Singapore,Singapore
1073,1,Tanglin,Singapore
1074,1,Raffles,Singapore
1075,1,Central Business District,Singapore


In [ ]:
# Create empty lists for lat and lng
list_lat = []
list_lng = []

In [ ]:
# Iterate over rows in dataframe\
for index, row in df_loc.iterrows(): 

    # Concatenate city and state column and convert to url encoding
    City = row['city']
    State = row['state']       
    city_query = str(City)+', '+str(State)
    url_city = requests.utils.quote(city_query)

    # Check the country id for each city
    if row["country_id"] == 1:
        country_code = "sg"
    elif row["country_id"] == 2:
        country_code = "ca"
    elif row["country_id"] == 3:
        country_code = "us"
    else:
        country_code = "au"

    # Combine into query url
    url = f"https://api.opencagedata.com/geocode/v1/json?key={api_key}&q={url_city}&countrycode={country_code}&limit=1&pretty=1"

    # Set Forward geocoding API request
    response = requests.get(url).json()

    # Try to get lat and lng, if error, return as NaN
    try:
        lat = response["results"][0]["geometry"]["lat"]
        lng = response["results"][0]["geometry"]["lng"]
    except:
        lat = lng = np.nan
    
    if index % 10 == 0:
        print(f"Row({index}): Logging {city_query} with lat: {lat} & lng: {lng}")

    list_lat.append(lat)
    list_lng.append(lng)

print("API data query done.")

In [ ]:
# Putting coordinate info into df
df_loc["lat"] = list_lat
df_loc["lng"] = list_lng

In [ ]:
df_loc

In [ ]:
# Export location df to csv for city location API use
df_loc.to_csv("US-Location.csv", index=False, header=True)

In [ ]:
import matplotlib.pyplot as plt
df_loc.plot(kind="scatter", x="lng", y="lat", alpha=0.4)
plt.show()